In [1]:
import cv2
import numpy as np
from keras.models import load_model
from statistics import mode
from utils.datasets import get_labels
from utils.inference import detect_faces
from utils.inference import draw_text
from utils.inference import draw_bounding_box
from utils.inference import apply_offsets
from utils.inference import load_detection_model
from utils.preprocessor import preprocess_input

USE_WEBCAM = True


cv2.namedWindow('window_frame')
video_capture = cv2.VideoCapture(0)

frame_window = 10
cam = None

# 감정
emotion_classifier = load_model('./models/emotion_model.hdf5')
emotion_labels = get_labels('fer2013')
emotion_target_size = emotion_classifier.input_shape[1:3]
emotion_window = []


def videoDetector(cam, cascade, age_net, gender_net, MODEL_MEAN_VALUES, age_list, gender_list,
                  emotion_classifier, emotion_labels, emotion_target_size):

    while True:
        # 이미지 불러오기
        ret, img = cam.read()

        # 영상 압축
        try:
            img1 = cv2.resize(img, dsize=None, fx=1.0, fy=1.0)  #for age, gender
        except:
            continue

        # 그레이 스케일 변환 
        gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY) #for age, gender
        gray2 = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #for emotion

        # cascade 얼굴 탐지 알고리즘
        results = cascade.detectMultiScale(gray1,
                                           scaleFactor=1.1,
                                           minNeighbors=5,
                                           minSize=(20, 20)
                                           )  #for age, gender
        faces = cascade.detectMultiScale(gray2,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(30, 30),
                                         flags=cv2.CASCADE_SCALE_IMAGE
                                         )#for emotion

        for box in results:
            x, y, w, h = box
            face = img[int(y):int(y + h), int(x):int(x + h)].copy() #for age, gender
            gray_face = gray2[y:y + h, x:x + w] #for emotion
            blob = cv2.dnn.blobFromImage(face, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

            try:
                gray_face = cv2.resize(gray_face, (emotion_target_size)) 
            except:
                continue

            # emotion
            gray_face = preprocess_input(gray_face, True)
            gray_face = np.expand_dims(gray_face, 0)
            gray_face = np.expand_dims(gray_face, -1)
            emotion_prediction = emotion_classifier.predict(gray_face)
            emotion_probability = np.max(emotion_prediction)
            emotion_label_arg = np.argmax(emotion_prediction)
            emotion_text = emotion_labels[emotion_label_arg]
            emotion_window.append(emotion_text)

            # gender
            gender_net.setInput(blob)
            gender_preds = gender_net.forward()
            gender = gender_preds.argmax()
            
            # age
            age_net.setInput(blob)
            age_preds = age_net.forward()
            age = age_preds.argmax()
            
            
            # 분석 결과
            info = f"{gender_list[gender]} {age_list[age]} {emotion_text}"
            
            # 화면 입력
            cv2.rectangle(img, (x, y), (x + w, y + h), (255, 255, 255), thickness=2)
            cv2.putText(img, info, (x, y - 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            
            # 콘솔 출력
            print(info)

        # 영상 출력
        cv2.imshow('facenet', img)

        if cv2.waitKey(1) & 0xFF == ord('q'):  
            break

# 얼굴 탐지 모델 가중치
cascade_filename = './models/haarcascade_frontalface_alt.xml'
# 모델 불러오기
cascade = cv2.CascadeClassifier(cascade_filename)


MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# 나이
age_net = cv2.dnn.readNetFromCaffe(
	'./models/deploy_age.prototxt',
	'./models/age_net.caffemodel')

# 성별
gender_net = cv2.dnn.readNetFromCaffe(
	'./models/deploy_gender.prototxt',
	'./models/gender_net.caffemodel')


age_list = ['(0 ~ 2)','(4 ~ 6)','(8 ~ 12)','(15 ~ 20)',
            '(25 ~ 32)','(38 ~ 43)','(48 ~ 53)','(60 ~ 100)']
gender_list = ['Male', 'Female']



if (USE_WEBCAM == True):
     cam = cv2.VideoCapture(0)  # Webcam source

else:
    cam = cv2.VideoCapture('./demo/dinner.mp4') 



videoDetector(cam,cascade,age_net,gender_net,MODEL_MEAN_VALUES,age_list,gender_list, emotion_classifier, emotion_labels, emotion_target_size)

cam.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 100ms/step
Female (15 ~ 20) fear
1/1 [==============================] - 0s 8ms/step
Female (15 ~ 20) fear
1/1 [==============================] - 0s 8ms/step
Female (15 ~ 20) fear
1/1 [==============================] - 0s 9ms/step
Female (15 ~ 20) surprise
1/1 [==============================] - 0s 10ms/step
Female (15 ~ 20) neutral
1/1 [==============================] - 0s 8ms/step
Female (15 ~ 20) neutral
1/1 [==============================] - 0s 9ms/step
Male (25 ~ 32) neutral
1/1 [==============================] - 0s 8ms/step
Female (25 ~ 32) neutral
1/1 [==============================] - 0s 9ms/step
Male (25 ~ 32) sad
1/1 [==============================] - 0s 8ms/step
Female (25 ~ 32) neutral
1/1 [==============================] - 0s 8ms/step
Male (25 ~ 32) neutral
1/1 [==============================] - 0s 8ms/step
Female (4 ~ 6) neutral
1/1 [==============================] - 0s 8ms/step
Female (15 ~ 20) neutral
1/1 [========================

KeyboardInterrupt: 